In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import transforms
from PIL import Image

# Define class names
class_names = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 
                'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']

class VGG_net(nn.Module):
    def __init__(self, input_channels, num_classes=2):
        super(VGG_net, self).__init__()
        self.in_channels = input_channels
        self.conv_layers = self.create_conv_layers([64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"])
        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        for layer in architecture:
            if type(layer) is int:
                out_channels = layer
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                    kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                            nn.BatchNorm2d(layer),
                            nn.ReLU()]
                in_channels = layer
            elif layer == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
        return nn.Sequential(*layers)

def predict_single_image(image_path, model, class_names):
    try:
        image = Image.open(image_path)
        data_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
        ])
        image_tensor = data_transform(image).unsqueeze(0)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        with torch.no_grad():
            model.eval()
            output = model(image_tensor)

        _, predicted_class = torch.max(output, 1)
        raw_predictions = output.squeeze().tolist()

        if raw_predictions[class_names.index('Cucumber')] > 0.7:
            print(f"This is Cucumber")
            return {
                'image_path': image_path,
                'predicted_class': 'Cucumber',
            }
        else:
            print(f"This is not Cucumber")
            return {
                'image_path': image_path,
                'predicted_class': class_names[predicted_class.item()],
            }

    except Exception as e:
        return {
            'image_path': image_path,
            'error': str(e)
        }

if __name__ == "__main__":
    # Load the trained model
    model_path = '../fun2/VGG16_torch_model_epoch40.pt'
    model = VGG_net(input_channels=3, num_classes=len(class_names))
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()

    image_path = '../testing-images/Unseen/1.jpg'
    result = predict_single_image(image_path, model, class_names)


This is Cucumber
